In [1]:
import pandas as pd
import os
from os import path
import json

In [2]:
from process_core import mainPageProcess

/home/mgsadmin/anaconda3/envs/crm_syd/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [19]:
fileid = "7bdb5eb1-d4fc-4ecd-890d-107a5e67a81d"

In [20]:
mp = mainPageProcess()

In [21]:
cbs_dict,cpl_dict,ccf_dict,meta_dict,final_notes_dict,ref_notes_list, notes_region_meta_data,cropped_table_dict = mp.process_main_pages(fileid=fileid)

In [36]:
mp.cropped_table_dict

{'a11dbe99-e39c-39bc-ae68-f9f8997515e5':                                                    0     1      2
 0                                                18.   NaN    NaN
 1                                In thousands of AUD  2022   2021
 2     Amount expected to be settled within 12 months    91     73
 3  Amount expected to be settled after more than ...   NaN    NaN
 4                                                NaN    91     73
 5  Reconciliation at the beginning and end of the...   NaN    NaN
 6                                    Opening Balance    73    177
 7            Transfer to included in opening balance  (73)  (177)
 8  Increase in cash received excluding amounts re...    91     73
 9                                    Closing balance    91     73,
 '35d47ae0-a69e-3dbc-b5bb-a0431943ca27':                           0                      1          2      3
 0                       NaN  Office Other Premises  Equipment  Total
 1   Balance at 1 April 2021              

In [35]:
len(mp.standardised_cropped_dict)

7

In [25]:
output_folder = './output/'
file_folder = os.path.join(output_folder,fileid)

In [26]:
try:
    os.mkdir(file_folder)
except:
    pass

In [27]:
##saving notes and respective account dict
with open(os.path.join(file_folder,f"{fileid}_notes_account_dict.json"),"w") as f:
    json.dump(mp.final_notes_dict,f)

In [28]:
##saving notes region meta_data
notes_region_meta_data.to_csv(os.path.join(file_folder,f"{fileid}_notes_region_meta_data.csv"),index=False)

In [29]:
###saving notes reference dict
with open(os.path.join(file_folder,f"{fileid}_notes_ref_list.json"),"w") as f:
    json.dump(mp.notes_ref_dict,f)

In [30]:
##saving main page data in excel
writer = pd.ExcelWriter(os.path.join(file_folder,f"{fileid}_notes_cropped_df.xlsx"), engine='xlsxwriter')
for key,value in mp.cropped_table_dict.items():
    value.to_excel(writer, sheet_name=key[:20],index=False)
writer.save()

In [31]:
## saving data in excel
writer = pd.ExcelWriter(os.path.join(file_folder,f"{fileid}_main_pages.xlsx"), engine='xlsxwriter')
for key,value in cbs_dict.items():
    value.to_excel(writer, sheet_name=f"cbs_{key}",index=False)
for key,value in cpl_dict.items():
    value.to_excel(writer, sheet_name=f"cpl_{key}",index=False)
for key,value in ccf_dict.items():
    value.to_excel(writer, sheet_name=f"ccf_{key}",index=False)
writer.save()

In [32]:
writer = pd.ExcelWriter(os.path.join(file_folder,f"{fileid}_notes_standard_cropped_df.xlsx"), engine='xlsxwriter')
for key,value in mp.standardised_cropped_dict.items():
    # print(key)
    # print(value)
    if value is not None:
        value.to_excel(writer, sheet_name=key[:20],index=False)
    else:
        temp_df = pd.DataFrame()
        temp_df.to_excel(writer, sheet_name=key[:20],index=False)
writer.save()

In [10]:
### testing code

In [6]:
from database import get_db, get_db1
import db_models

In [7]:
db = get_db1()

In [8]:
page_query = db.query(db_models.PageLogs).filter(db_models.PageLogs.fileid == "599060e9-8b46-46ec-87b9-c79cde8a82b7").order_by(db_models.PageLogs.time.desc())
pages = page_query.all()

In [9]:
for page in pages:
    if page.page_number == 7:
        tabale_query = db.query(db_models.TableLogs).filter(db_models.TableLogs.pageid == page.pageid).order_by(db_models.TableLogs.time.desc())
        tables = tabale_query.all()
        for table in tables:
            print(pd.read_html(table.html_string)[0])

                                             0      1         2         3
0                                          NaN    NaN      2020      2019
1                                          NaN    NaN      $000      $000
2                                          NaN  Notes       NaN       NaN
3                                       Assets    NaN       NaN       NaN
4                               Current assets    NaN       NaN       NaN
5                    Cash and cash equivalents      6     19589     28259
6                  Trade and other receivables      7     13624     11999
7                                  Inventories      8     27198     26953
8                             Financial assets      9       NaN     29753
9   Income tax receivable from related parties     20      6318      7530
10                        Other current assets     10       489       328
11                        Total current assets    NaN     67218    104822
12                          Non-curren